<a href="https://colab.research.google.com/github/Maxibrionest/FMY-Fundamentos-Machine-Learning/blob/main/Archivo_2_4_5/2_4_5_Clasificacion_Regresion_Logistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción 
 

# Regresión Logística - Validación
$$y = \beta_0 + x_1\beta_1 + ... + x_k\beta_k$$

$$p = f(y)= \frac{1}{1+e^{-y}} = \frac{1}{1+e^{-(\beta_0 + x_1\beta_1 + ... + x_k\beta_k)}}$$
 


 # Actividades 
 ESTABILIDAD DE UNA MATRIZ ELÉCTRICA

La estabilidad en una matriz energética del país es de suma importancia para el progreso económico y el bienestar de sus habitantes (solo tener presente la sensación de inseguridad, malestar, etc. que se puede generar con un corte no previsto puede afectar el mercado económico y el bienestar de una población)

El desafio es tratar de determinar si la red es estable (columna ```stabf```) basado en un conjunto de caracteristicas disponibles:


1. tau[x]: reaction time of participant (real from the range [0.5,10]s). Tau1 - the value for electricity producer. 
2. p[x]: nominal power consumed(negative)/produced(positive)(real). For consumers from the range [-0.5,-2]s^-2; p1 = abs(p2 + p3 + p4) 
3. g[x]: coefficient (gamma) proportional to price elasticity (real from the range [0.05,1]s^-1). g1 - the value for electricity producer. 
4. stab: the maximal real part of the characteristic equation root (if positive - the system is linearly unstable)(real) 
5. stabf: the stability label of the system (categorical: stable/unstable) 


Estos datos fueron usados en el paper: "Towards Concise Models of Grid Stability"

https://archive.ics.uci.edu/ml/datasets/Electrical+Grid+Stability+Simulated+Data+

¡Empezamos! 



## Cargamos el dataset

In [1]:
import pandas as pd
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00471/Data_for_UCI_named.csv')

df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [2]:
df.isna().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

In [3]:
df.stabf.unique()

array(['unstable', 'stable'], dtype=object)

In [19]:
df.stabf.value_counts()/df.shape[0]

unstable    0.638
stable      0.362
Name: stabf, dtype: float64

## Preparamos el dataset

¿Por qué hay variables que se borran?

Explica la celda siguiente...

In [5]:
X = df.drop(['stab','stabf'],axis=1)
Y = df.stabf.copy()

 - Del dataset obtendremos un modelo para determinar ***stab*** que a su vez, según su signo determina ***stabf***. Entonces, para poder entrenarlo, debemos sacar a los valores target (***stab*** y ***stabf***) del dataset. Luego copiamos ***stabf*** a *Y*, que utilizaremos como target.

## Cargamos y Ajustamos el modelo

In [7]:
import numpy as np
from sklearn.linear_model import LogisticRegression

mo1 = LogisticRegression(solver='lbfgs')

mo1.fit(X, Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [8]:
Yhat = mo1.predict(X)
Yprob = mo1.predict_proba(X)

In [9]:
Yhat

array(['unstable', 'unstable', 'unstable', ..., 'stable', 'unstable',
       'unstable'], dtype=object)

In [12]:
Yprob

array([[0.00305861, 0.99694139],
       [0.15841728, 0.84158272],
       [0.30149768, 0.69850232],
       ...,
       [0.85798006, 0.14201994],
       [0.02217816, 0.97782184],
       [0.10345702, 0.89654298]])

## Evaluamos los resultados

Explica que significan cada una de las métricas siguientes, en cuanto al valor que están mostrando.

In [13]:
from sklearn.metrics import classification_report
print(classification_report(Y, Yhat))

              precision    recall  f1-score   support

      stable       0.77      0.70      0.73      3620
    unstable       0.84      0.88      0.86      6380

    accuracy                           0.82     10000
   macro avg       0.80      0.79      0.80     10000
weighted avg       0.81      0.82      0.81     10000



 - Se utilizó [esta página web](https://blog.exsilio.com/all/accuracy-precision-recall-f1-score-interpretation-of-performance-measures/#:~:text=F1%20score%20-%20F1%20Score%20is,have%20an%20uneven%20class%20distribution.) como fuente.

 - Accuracy es la cantidad de veces que el modelo estuvo de acuerdo con la realidad. En este caso, el 82% de las predicciones fueron correctas.

 - Presicion es la relación entre los datos clasificados correctamente, entre los observados positivamente y los clasificados positivamente. Alta presicion, bajo falsos positivos. En otras palabras, que tanto de los que el modelo clasifica como positivos son positivos en las observaciones. En la talb anterior: de todos los predichos como ***stable***, 77% lo eran en las observaciones. Lo mismo para ***unstable*** con un 84%.

 - Recall o exhaustividad - sensitividad, es la razón entre la cantidad de observaciones positivas (***stable***) y las que fueron clasificadas como positivas. Responde a "De todos los casos con ***stable***, ¿Cuántos fueron clasificados como tal?". De la tabla, del total de obseravciones ***stable***, un 70% fue capturado por el modelo.

 - F1 score es el promedio ponderado entre Presicion y Recall. Para modelos en los que los falsos positivos y falsos negativos tienen valores e impactos distintos, lo mejor es usar F1 score en vez de Accuracy.

Explica que está indicando la matriz de confusión siguiente

In [14]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y, Yhat)

array([[2549, 1071],
       [ 771, 5609]])

In [17]:
confusion_matrix(Y, Yhat)

array([[2549, 1071],
       [ 771, 5609]])

La matriz de confusión sigue esta forma:
 
 |   ---         | Real     |          |
 | ---           | :---:    | :---:    |
 | **Predicho**  | Unstable | Stable   |
 | Unstable      | 2549     | 1071     |
 | Stable        | 771      | 5609     |

 La tabla indica que respecto a los datos predichos tenemos:
  - 2549 Instables correctos y 5609 Estables correctos (*Accuracy*)
  - 

In [ ]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
Y1 = enc.fit_transform(Y)
Yhat1 = enc.fit_transform(Yhat)


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y1,Yhat1)

0.7916486257122568

In [ ]:
print(Yhat[0:10])
print(Yhat1[0:10])
print(Yprob[0:10,])

['unstable' 'unstable' 'unstable' 'unstable' 'unstable' 'unstable'
 'unstable' 'unstable' 'stable' 'unstable']
[1 1 1 1 1 1 1 1 0 1]
[[0.00305861 0.99694139]
 [0.15841728 0.84158272]
 [0.30149768 0.69850232]
 [0.26785973 0.73214027]
 [0.01479175 0.98520825]
 [0.48347719 0.51652281]
 [0.35168981 0.64831019]
 [0.18119779 0.81880221]
 [0.97979887 0.02020113]
 [0.05315412 0.94684588]]


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y1,Yprob[:,1])

0.8911449799961897

## Será necesario escalar los datos?

Esto se hace ya que las características son completamente distintas en magnitudes, unidades y rango por lo que lo mejor es escalarlos para llevarlos a un mismo nivel de magnitudes.

Para realizar este procedimiento importaremos el método StandarScaler.

INVESTIGA!....¿por qué es necesario ESCALAR? ¿cómo funciona el método StandardScaler? ¿y el MinMaxScaler? ¿Hay otros? ¿Cuáles?

####Escalamiento (opción 1)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
Xscaled = scaler.transform(X)


¿Hubo algún cambio en la verificación de las métricas?...revisa las siguientes celdas....

In [ ]:
mo1 = LogisticRegression(solver='lbfgs')
mo1.fit(Xscaled, Y)
Yhat = mo1.predict(Xscaled)

print(classification_report(Y, Yhat))

print(confusion_matrix(Y, Yhat))

              precision    recall  f1-score   support

      stable       0.77      0.71      0.74      3620
    unstable       0.84      0.88      0.86      6380

    accuracy                           0.82     10000
   macro avg       0.80      0.79      0.80     10000
weighted avg       0.81      0.82      0.81     10000

[[2555 1065]
 [ 771 5609]]


####Escalamiento (opción 2)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scalerN = MinMaxScaler()
scalerN.fit(X)
XscaledN = scalerN.transform(X)

mo1 = LogisticRegression(solver='lbfgs')
mo1.fit(XscaledN, Y)
Yhat = mo1.predict(XscaledN)

print(classification_report(Y, Yhat))

print(confusion_matrix(Y, Yhat))

              precision    recall  f1-score   support

      stable       0.77      0.70      0.74      3620
    unstable       0.84      0.88      0.86      6380

    accuracy                           0.82     10000
   macro avg       0.80      0.79      0.80     10000
weighted avg       0.81      0.82      0.81     10000

[[2549 1071]
 [ 765 5615]]


¿Hubo algún cambio en las métricas? ¿ qué indican estas?

## Evitando el Overfitting

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y,test_size=0.2)

## Re-entrenamos y evaluamos los resultados

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression

mo1 = LogisticRegression(solver='lbfgs')

mo1.fit(Xtrain, Ytrain)

Yhat = mo1.predict(Xtest)
Yprob = mo1.predict_proba(Xtest)

from sklearn.metrics import classification_report
print(classification_report(Ytest, Yhat))

              precision    recall  f1-score   support

      stable       0.80      0.70      0.74       746
    unstable       0.83      0.89      0.86      1254

    accuracy                           0.82      2000
   macro avg       0.81      0.80      0.80      2000
weighted avg       0.82      0.82      0.82      2000



Explica cada una de las métricas anteriores. ¿Qué ocurre con el modelo?

## Sintonización del Modelo

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

params = {"max_iter": [50,100,150,200],
          "solver": ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
         }
grid = GridSearchCV(estimator=model, param_grid=params,cv=5,scoring='accuracy')
grid.fit(Xtrain, Ytrain,)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'max_iter': [50, 100, 150, 200],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [ ]:
print(grid.best_score_)
print(grid.best_params_)

0.8140000000000001
{'max_iter': 100, 'solver': 'saga'}


In [ ]:
pd.DataFrame(grid.cv_results_).iloc[grid.best_index_]

mean_fit_time                                   0.192373
std_fit_time                                 0.000756647
mean_score_time                               0.00789261
std_score_time                                  0.002195
param_max_iter                                       100
param_solver                                        saga
params               {'max_iter': 100, 'solver': 'saga'}
split0_test_score                                0.81875
split1_test_score                                   0.81
split2_test_score                                 0.8075
split3_test_score                                  0.815
split4_test_score                                0.81875
mean_test_score                                    0.814
std_test_score                                0.00456892
rank_test_score                                        1
Name: 9, dtype: object

In [ ]:
Yhat = grid.predict(Xtest)
Yprob = grid.predict_proba(Xtest)

from sklearn.metrics import classification_report
print(classification_report(Ytest, Yhat))

              precision    recall  f1-score   support

      stable       0.80      0.70      0.74       746
    unstable       0.83      0.89      0.86      1254

    accuracy                           0.82      2000
   macro avg       0.81      0.80      0.80      2000
weighted avg       0.82      0.82      0.82      2000



## CONCLUSIONES

¿Es este el mejor modelo? ¿Por qué?

Reflexiona sobre el ejercicio y realiza una explicación que puedas compartir con el curso.